In [1]:
import os
import pandas as pd
import mrigutilities as mu
import mrigstatics
import datetime
path = "F:\\Mrig Analytics\\mediafiles\\downloads\\equity_options_futures.csv"

f = pd.read_csv(path)
f

,portfolio_name,position_date,symbol,security_type,underlying,strike,option_type,expiry_date,quantity,price
0,SB,5/7/2024,BANKNIFTY24JUL47500PE,OPT,BANKNIFTY,47500.0,PE,7/25/2024,-45,672.95
1,SB,5/7/2024,BANKNIFTY24JUL48000PE,OPT,BANKNIFTY,48000.0,PE,7/25/2024,-60,830.54
2,SB,5/7/2024,BANKNIFTY24JUL48500PE,OPT,BANKNIFTY,48500.0,PE,7/25/2024,-45,977.65
3,SB,5/7/2024,BANKNIFTY24JUN47500CE,OPT,BANKNIFTY,47500.0,CE,6/27/2024,-30,1975.00
4,SB,5/7/2024,BANKNIFTY24JUN48000CE,OPT,BANKNIFTY,48000.0,CE,6/27/2024,-120,1871.50
5,SB,5/7/2024,BANKNIFTY24JUN48500CE,OPT,BANKNIFTY,48500.0,CE,6/27/2024,-90,1474.67
6,SB,5/7/2024,BANKNIFTY24JUN49000CE,OPT,BANKNIFTY,49000.0,CE,6/27/2024,-30,1427.50
7,SB,5/7/2024,BANKNIFTY24JUN49500CE,OPT,BANKNIFTY,49500.0,CE,6/27/2024,-45,1259.33
8,SB,5/7/2024,BANKNIFTY24JUNFUT,FUT,BANKNIFTY,NaN,NaN,6/27/2024,120,49520.31
9,SB,5/7/2024,NIFTY2450923000CE,OPT,NIFTY,23000.0,CE,5/9/2024,375,61.23


In [52]:
user='santoshbag'
name = 'SB'
def show_portfolio(name,user):
    mrig_engine = mu.sql_engine(dbname=mrigstatics.MRIGWEB[mrigstatics.ENVIRONMENT])
    rb_engine = mu.sql_engine(dbname=mrigstatics.RB_WAREHOUSE[mrigstatics.ENVIRONMENT])
    
    portfolio = pd.read_sql(
        "select portfolio_name as portfolio_name,	position_date as position_date,trim(SPLIT_PART(sec_id,'|',1)) as security,	\
        security_type as type,		quantity as qty,	price as cost from portfolio where \
        account_id='" + user + "' and portfolio_name='" + name + "'",mrig_engine)
    # return portfolio
    # print(portfolio)

    port_analytics = pd.read_sql("select * from analytics where date = (select max(date) from analytics)",rb_engine)
    port_analytics.rename(columns={'date':'valuation_date_time','secid':'security'},inplace=True)
    port_analytics = pd.merge(portfolio,port_analytics,how='left',on='security')
    port_analytics['delta'] = port_analytics['delta'] * port_analytics['qty']
    port_analytics['gamma'] = port_analytics['gamma'] * port_analytics['qty']
    port_analytics['theta_per_day'] = port_analytics['theta_per_day'] * port_analytics['qty']
    # port_analytics.dropna(inplace=True)
    print(port_analytics[['portfolio_name','valuation_date_time',  'position_date', 'security', 'type', 'qty', 'cost',
       'price', 'delta', 'gamma', 'theta_per_day', 'vega', 'rho',       'analytic_value']])    
    
show_portfolio(name,user)

   portfolio_name        valuation_date_time position_date  \
0              SB 2024-05-13 21:39:22.014523    2024-05-07   
1              SB 2024-05-13 21:39:22.014523    2024-05-07   
2              SB 2024-05-13 21:39:22.014523    2024-05-07   
3              SB 2024-05-13 21:39:22.014523    2024-05-07   
4              SB 2024-05-13 21:39:22.014523    2024-05-07   
5              SB 2024-05-13 21:39:22.014523    2024-05-07   
6              SB 2024-05-13 21:39:22.014523    2024-05-07   
7              SB 2024-05-13 21:39:22.014523    2024-05-07   
8              SB 2024-05-13 21:39:22.014523    2024-05-07   
9              SB 2024-05-13 21:39:22.014523    2024-05-07   
10             SB 2024-05-13 21:39:22.014523    2024-05-07   
11             SB 2024-05-13 21:39:22.014523    2024-05-07   
12             SB 2024-05-13 21:39:22.014523    2024-05-07   
13             SB 2024-05-13 21:39:22.014523    2024-05-07   
14             SB 2024-05-13 21:39:22.014523    2024-05-07   
15      

In [49]:
'''
Analytics for portfolio
Algorithm

1. Get the list of all symbols in portfolios
2. Create objects out of those 
3. Run Analytics on each
4. Store Analytics in database
5. Aggregate analytics at portfolio level
6. Display Analytics as table or as graphs  

'''
import kite.kite_account as ka
import strategies.marketoptions as mo
import strategies.marketfutures as mfut
import json

kite_object = ka.kite_account()
ssn = kite_object.session

def run_portfolio_analtyics():
    mrig_engine = mu.sql_engine(dbname=mrigstatics.MRIGWEB[mrigstatics.ENVIRONMENT])
    rb_engine = mu.sql_engine(dbname=mrigstatics.RB_WAREHOUSE[mrigstatics.ENVIRONMENT])
    
    symbols = mrig_engine.execute("select distinct split_part(sec_id,'|',1) as symbol from portfolio").fetchall()
    symbols = str([s[0] for s in symbols]).replace('[','(').replace(']',')')
    print(symbols)
    sec_m = pd.read_sql(" select symbol,security_type,underlying,strike,option_type,expiry_date from security_master where symbol in "+symbols,rb_engine)
    sec_m['underlying_price'] = sec_m['underlying'].apply(lambda x : mu.price(x))
    sec_m['cmp'] = sec_m[['symbol','expiry_date']].apply(lambda x : ssn.quote('NFO:'+x['symbol'])['NFO:'+x['symbol']]['last_price'] \
                                                         if (x['expiry_date'] >= datetime.date.today()) else -1,axis=1)

    sec_m['obj'] = sec_m[['symbol','expiry_date','security_type','strike','option_type','cmp','underlying']] \
    .apply(lambda x : (mo.MarketOptions(x['underlying'], float(x['strike']), x['expiry_date'],  x['option_type']) \
                       if x['security_type'] == 'OPT' else mo.MarketFutures(x['underlying'], float(x['cmp']), x['expiry_date'])) \
                       if (x['expiry_date'] >= datetime.date.today()) else None, axis=1)
    sec_m['analytics'] = sec_m[['cmp','obj','expiry_date']] \
    .apply(lambda x : x['obj'].valuation(x['cmp']) if (x['expiry_date'] >= datetime.date.today()) else {'NPV': 0, 'delta': 0, \
                     'gamma': 0, 'theta': 0, 'theta_per_day': 0,'rho': 0, 'vega': 0, 'strike_sensitivity': 0, 'dividendRho': 0, \
                     'volatility': 0}, axis=1)
    sec_m['date'] = datetime.datetime.now()
    
    print(sec_m)
    return sec_m

def populate_portfolio_analytics(analytics_db):
    rb_engine = mu.sql_engine(dbname=mrigstatics.RB_WAREHOUSE[mrigstatics.ENVIRONMENT])    
    df = pd.DataFrame()
    analytics_map = {
        'symbol' : 'secid',
        'cmp' : 'price',
        'NPV' : 'analytic_value'
    }
    symbols = analytics_db['symbol']
    for sym in symbols:
        analytics = analytics_db[analytics_db['symbol'] == sym]
        result = analytics['analytics'].values[0] if len(analytics['analytics'].values) > 0 else None
        analytics['others'] = json.dumps(result)
        for key, val in result.items():
            analytics[key] = val
        analytics.rename(columns=analytics_map,inplace=True)
        analytics = analytics[['date','secid','price','delta','gamma','theta_per_day','vega','rho','others','analytic_value']]
        df = pd.concat([df,analytics])
        # print(analytics)
    df.to_sql('analytics',rb_engine,index=False,if_exists='append')
    print(df)


sec = run_portfolio_analtyics()
populate_portfolio_analytics(sec)

connection active
('NIFTY24JUL22000PE', 'NIFTY24JUN20500CE', 'NIFTY2450923000CE', 'BANKNIFTY24JUNFUT', 'NIFTY24MAY23000CE', 'NIFTY24JUL22200PE', 'BANKNIFTY24JUL48500PE', 'NIFTY24JUN21000CE', 'NIFTY24JUL21500CE', 'BANKNIFTY24JUN49000CE', 'BANKNIFTY24JUN48000CE', 'NIFTY24JUN22300PE', 'NIFTY24JUN22500PE', 'BANKNIFTY24JUL47500PE', 'BANKNIFTY24JUN48500CE', 'NIFTY24JUN22800CE', 'NIFTY24JUNFUT', 'BANKNIFTY24JUL48000PE', 'NIFTY24JUN20000CE', 'BANKNIFTY24JUN47500CE', 'NIFTY24APRFUT', 'BANKNIFTY24APRFUT', 'NIFTY24MAY22800CE', 'NIFTY24APR22800CE', 'BANKNIFTY24JUN49500CE', 'NIFTY24JUN22300CE')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

{'day_count': <QuantLib.QuantLib.Thirty360; proxy of <Swig Object of type 'QuantLib::Thirty360 *' at 0x00000216E713FDB0> >, 'calendar': <QuantLib.QuantLib.India; proxy of <Swig Object of type 'QuantLib::India *' at 0x00000216E713FCC0> >, 'compounding': 1, 'compounding_frequency': 1, 'interpolation': <QuantLib.QuantLib.Linear; proxy of <Swig Object of type 'Linear *' at 0x00000216E713FE70> >, 'shiftparameter': None}
Implied Vol --> 0.2045918120865112
{'day_count': <QuantLib.QuantLib.Thirty360; proxy of <Swig Object of type 'QuantLib::Thirty360 *' at 0x00000216E713FDB0> >, 'calendar': <QuantLib.QuantLib.India; proxy of <Swig Object of type 'QuantLib::India *' at 0x00000216E713FCC0> >, 'compounding': 1, 'compounding_frequency': 1, 'interpolation': <QuantLib.QuantLib.Linear; proxy of <Swig Object of type 'Linear *' at 0x00000216E713FE70> >, 'shiftparameter': None}
Implied Vol --> 0.1974474192162945
{'day_count': <QuantLib.QuantLib.Thirty360; proxy of <Swig Object of type 'QuantLib::Thirty3

C:\Users\santoshbag\AppData\Local\Temp\ipykernel_21936\3391838425.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analytics['others'] = json.dumps(result)
C:\Users\santoshbag\AppData\Local\Temp\ipykernel_21936\3391838425.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analytics[key] = val
C:\Users\santoshbag\AppData\Local\Temp\ipykernel_21936\3391838425.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

In [ ]:
def show_portfolio_analytics(accountid):
    

In [17]:
dic = sec[sec['symbol'].str.contains('BANKNIFTY24JUL48500PE')]['analytics'].values[0]
for key, val in dic.items():
    print(key,val)

NPV 2220.1930542757473
delta -0.5517117714075682
gamma 8.85984038249146e-05
theta -2948.4607426686875
theta_per_day -8.07797463744846
rho -5888.326467964356
vega 8330.712376579391
strike_sensitivity 0.5827622071387409
dividendRho 5425.786248323576
volatility 0.2109807887884335
Underlying 47205.3984375
